##Develop Pipeline for monthly processing

For month_nn_, where _nn_=1..72. Steps are:
1. chdir to, creating if necessary, a folder month_nn_ under folder P301\_monthly, which contains the basic config.txt file. Copy the appropriate seed\_pgw_nn_.txt
2. run stage monthly: this creates the model from the 6-year version, then runs seedcheck\_pgw.
Specifically, it initializes process.Process with the flag fix_spectrum_flag=True. 
This fixes the spectra for the first iteration only. Need to also set the model so that these sources, from the 6-year final model, keep the same spectra.
Note, need to start update_seeds, but currently need the plots seedcheck 
3. run update_seeds to incorporate successful seeds -- this triggers a full iteration cycle (but avoid final)
4. run tables

In [38]:
%matplotlib inline
os.chdir(os.path.expandvars('$FERMI/skymodels/P301_monthly/month13'))
from uw.like2.analyze import (seedcheck, ); 
from uw.like2 import process; reload(process)
map(reload, (seedcheck, process));

### Test code for creating a new monthly model

In [39]:
r = process.Process('.', fix_spectra_flag=True)

Using configuration file "../config.txt" in folder: /nfs/farm/g/glast/g/catalog/pointlike/skymodels/P301_monthly/month13/.
Opening dataset P301_monthxx from key in /afs/slac/g/glast/groups/catalog//pointlike/fermi/data/dataspec.py
invoking Data.setPhotonBinner for front/back event types...
Extracting DSS from existing binfile
GTI from binfile Gti: 532 intervals from 270890005 to 273519993, on time 2161635
Verified binfile /afs/slac/g/glast/groups/catalog//pointlike/fermi/data/P8_P301/binfiles/P301_Source_013_zmax100_4bpd.fits
no header info in ltcube?
Accepting ltcube without DSS info since legacy specified
Failed gti check:
  ltcube: Gti: 525 intervals from 270890000 to 273520000, on time 2161857 
 binfile: Gti: 532 intervals from 270890005 to 273519993, on time 2161635
loading binfile /afs/slac/g/glast/groups/catalog//pointlike/fermi/data/P8_P301/binfiles/P301_Source_013_zmax100_4bpd.fits ... found 2239987 photons in 40 bands, energies 10-1000000 MeV
dataset "P301_monthxx", found in 

In [40]:
r.setup_roi(840); r.process()

288 total sources: 4 extended, 3 global
2015-07-28 13:09:31 - HP12_0840 - interactive
------------------------------------------------------------------------------------------ 
	 Nearby sources within 5.0 degrees before fit, logL=4344
name               dist        ra       dec        TS eflux(eV)    index   energy   beta/b
P86Y4099            0.1   268.472   -25.642        16      21.7!    1.87     2332     0.35 
P967-445            0.6   268.972   -25.258        -0       6.8*    2.17     2082     0.31 
P86Y4078            1.7   267.011   -24.781        15      38.5!    1.98     1346     0.34 
S966-1415           1.8   266.574   -25.696        -1       6.8*    2.41     3005     0
PSR J1801-2451      1.9   270.404   -24.887         0       4.9*    1.13     6278     1
P86Y4115            1.9   269.664   -24.076        -3      11.3*    2.17     5394     0
P86Y4060            2.2   266.259   -25.128        -4      13.7*    2.96     1092     0
S970-936            2.2   270.239   -24.137  

In [27]:
for src in r.free_sources:
    m=src.model
    for i,parname in enumerate(m.param_names[1:]):
        if m.free[i+1]:
            r.freeze(parname, src.name)
    m.fixed_spectrum=True

In [28]:
r.summarize()

current likelihood, est. diff to peak: 4344.0, 9.04
Name                  index     value  error(%)  gradient
S971-1693_Norm            0 8.312e-13      17.6      16.0
P86Y4099_Norm             1 1.319e-12       9.7      -1.0
P967-445_Norm             2  4.92e-13      28.4       1.0
P86Y4073_Norm             3 2.785e-12      10.8      14.1
P86Y4056_Norm             4 5.561e-14      26.1       0.2
S971-0807_Norm            5 1.492e-14      42.8       3.3
S971-0772_Norm            6 2.077e-13      31.9       5.7
S966-1587_Norm            7 4.012e-14      26.8       0.2
W28_Norm                  8 4.425e-11       2.5      86.4
S966-1817_Norm            9 8.814e-14      19.6       4.3
P86Y4115_Norm            10 4.873e-14      25.8       4.2
P86Y4078_Norm            11 7.137e-12       5.0      12.1
P86Y4119_Norm            12 8.599e-14      20.7       3.3
P86Y4136_Norm            13 9.757e-14      27.5       1.7
PSR J1801-2451_Norm      14 2.391e-13      59.6       0.3
P967-336_Norm       

In [29]:
r.fit()

Minimizer warning: bad errors for values ['S966-1587_Norm' 'W28_Norm' 'P86Y4119_Norm' 'S970-845_Norm']
34 calls, function value, improvement, quality: 4356.6, 12.58, 0.00
Name                  index     value  error(%)  gradient
S971-1693_Norm            0 5.781e-13     107.3       0.2
P86Y4099_Norm             1 1.499e-12      51.9       0.0
P967-445_Norm             2 1.221e-13    1232.8      -0.0
P86Y4073_Norm             3 1.416e-12     121.4      -0.0
P86Y4056_Norm             4 1.001e-13     144.4      -0.1
S971-0807_Norm            5 6.383e-16    1042.2       0.0
S971-0772_Norm            6 2.281e-13     136.1       0.0
S966-1587_Norm            7 5.896e-14       ***      -0.0
W28_Norm                  8 3.943e-11       ***       0.7
S966-1817_Norm            9 2.337e-14    1125.4      -0.1
P86Y4115_Norm            10 1.553e-15    1591.7       0.0
P86Y4078_Norm            11 5.536e-12      42.5       0.2
P86Y4119_Norm            12 7.058e-14       ***      -0.0
P86Y4136_Norm    

In [30]:
r.print_summary()

------------------------------------------------------------------------------------------ 
	 Nearby sources within 5.0 degrees HP12_0840
name               dist        ra       dec        TS eflux(eV)    index   energy   beta/b
P86Y4099            0.1   268.472   -25.642        17      24.6*    1.87     2332     0.35 
P967-445            0.6   268.972   -25.258         0       1.7*    2.17     2082     0.31 
P86Y4078            1.7   267.011   -24.781        19      29.9*    1.98     1346     0.34 
S966-1415           1.8   266.574   -25.696         0       2.6*    2.41     3005     0
PSR J1801-2451      1.9   270.404   -24.887         1       4.9*    1.13     6278     1
P86Y4115            1.9   269.664   -24.076        -0       0.4*    2.17     5394     0
P86Y4060            2.2   266.259   -25.128        -0       0.1*    2.96     1092     0
S970-936            2.2   270.239   -24.137         0       9.3*    2.03     8014     0
P86Y4119            2.2   269.805   -23.821         3  

In [35]:
r = process.Process('.', seed_key='pgw', quiet=True)

invoking Data.setPhotonBinner for front/back event types...
processing cuts:  ZENITH_ANGLE zenith_cut
ft1_cut DSTYP3: ZENITH_ANGLE
DSUNI3: deg
DSVAL3: 0:100
DSREF3: None
processing cuts:  THETA theta_cut
ft1_cut None
processing cuts:  EVENT_CLASS event_class_cut
ft1_cut None


In [41]:
r.setup_roi(601)

20 total sources: 0 extended, 3 global


In [42]:
reload(process)
process.seeds(r, 'pgw')

no seeds in ROI
